# Improved Transformer Sequence Training (V2)

Training improved version of Transformer Sequence model:
- Increased d_model: 128 → 256
- Increased num_layers: 4 → 6
- Increased dim_feedforward: 512 → 1024
- Batch-first format for better performance
- Layer normalization in classifier head
- Better training: 100-120 epochs, warmup, cosine annealing, gradient clipping


In [1]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json

PROJECT_ROOT = Path('/Volumes/SSanDisk/SpeechRec-German/without_context_windows')
sys.path.insert(0, str(PROJECT_ROOT))

from models.sequence.transformer_sequence_v2 import TransformerSequence_V2
from utils.training_utils import train_model, evaluate_model, WarmupCosineScheduler, LabelSmoothingCrossEntropy
from utils.data_loader import load_data, create_dataloaders

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS device")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict = load_data(PROJECT_ROOT)
dataloaders = create_dataloaders(df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict, batch_size=64)

train_loader = dataloaders['sequence']['train']
val_loader = dataloaders['sequence']['val']
test_loader = dataloaders['sequence']['test']

OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models' / 'improved_models'
class_weights = torch.tensor([class_weights_dict.get('0', class_weights_dict.get(0, 1.0)), 
                              class_weights_dict.get('1', class_weights_dict.get(1, 1.0))], dtype=torch.float32).to(device)


Using MPS device
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'delta_mfcc

## Model: Improved Transformer Sequence V2


In [2]:
model = TransformerSequence_V2(
    input_dim=128, 
    d_model=256, 
    nhead=8, 
    num_layers=6, 
    dim_feedforward=1024, 
    dropout=0.1, 
    num_classes=2,
    batch_first=True
).to(device)

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model: {model.get_config()['model_type']}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Loss function with label smoothing
criterion = LabelSmoothingCrossEntropy(smoothing=0.1, weight=class_weights)

# Optimizer with weight decay
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

# Learning rate scheduler with warmup and cosine annealing
num_epochs = 110
warmup_epochs = 10
scheduler = WarmupCosineScheduler(optimizer, warmup_epochs=warmup_epochs, total_epochs=num_epochs, min_lr=1e-6)

save_dir = OUTPUT_DIR / 'transformer_sequence_v2'
save_dir.mkdir(parents=True, exist_ok=True)

print(f"\nTraining configuration:")
print(f"- Epochs: {num_epochs}")
print(f"- Warmup epochs: {warmup_epochs}")
print(f"- Initial LR: {optimizer.param_groups[0]['lr']}")
print(f"- Label smoothing: 0.1")
print(f"- Gradient clipping: 1.0")
print(f"- Early stopping patience: 15")


Model: TransformerSequence_V2
Total parameters: 4,879,554
Trainable parameters: 4,879,554

Training configuration:
- Epochs: 110
- Warmup epochs: 10
- Initial LR: 0.0001
- Label smoothing: 0.1
- Gradient clipping: 1.0
- Early stopping patience: 15


In [3]:
history, best_epoch = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler,
    device, num_epochs=num_epochs, save_dir=save_dir, model_name='transformer_sequence_v2', 
    early_stopping_patience=15, max_grad_norm=1.0
)

checkpoint = torch.load(save_dir / 'best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
test_metrics, _, _, _ = evaluate_model(model, test_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics, f, indent=2)

print(f"\n{'='*60}")
print(f"Final Test Results:")
print(f"{'='*60}")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"F1-score: {test_metrics['f1']:.4f}")
print(f"ROC-AUC: {test_metrics['roc_auc']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"Best epoch: {best_epoch}")



Epoch 1/110
--------------------------------------------------


Train Loss: 0.5597, Train Acc: 0.7582
Val Loss: 0.7299, Val Acc: 0.6988
Val F1: 0.7081, Val ROC-AUC: 0.9346
Learning Rate: 0.000010
✓ New best model saved! (F1: 0.7081)

Epoch 2/110
--------------------------------------------------


Train Loss: 0.4262, Train Acc: 0.8745
Val Loss: 0.3879, Val Acc: 0.8926
Val F1: 0.8947, Val ROC-AUC: 0.9586
Learning Rate: 0.000020
✓ New best model saved! (F1: 0.8947)

Epoch 3/110
--------------------------------------------------


Train Loss: 0.4127, Train Acc: 0.8801
Val Loss: 0.3804, Val Acc: 0.8868
Val F1: 0.8897, Val ROC-AUC: 0.9628
Learning Rate: 0.000030

Epoch 4/110
--------------------------------------------------


Train Loss: 0.4075, Train Acc: 0.8873
Val Loss: 0.3748, Val Acc: 0.8930
Val F1: 0.8956, Val ROC-AUC: 0.9654
Learning Rate: 0.000040
✓ New best model saved! (F1: 0.8956)

Epoch 5/110
--------------------------------------------------


...

Train Loss: 0.2468, Train Acc: 0.9936
Val Loss: 0.3519, Val Acc: 0.9503
Val F1: 0.9503, Val ROC-AUC: 0.9822
Learning Rate: 0.000004

Epoch 99/110
--------------------------------------------------


Train Loss: 0.2457, Train Acc: 0.9945
Val Loss: 0.3539, Val Acc: 0.9498
Val F1: 0.9497, Val ROC-AUC: 0.9821
Learning Rate: 0.000004

Epoch 100/110
--------------------------------------------------


Train Loss: 0.2466, Train Acc: 0.9938
Val Loss: 0.3534, Val Acc: 0.9500
Val F1: 0.9499, Val ROC-AUC: 0.9820
Learning Rate: 0.000003

Epoch 101/110
--------------------------------------------------


Train Loss: 0.2463, Train Acc: 0.9942
Val Loss: 0.3516, Val Acc: 0.9513
Val F1: 0.9512, Val ROC-AUC: 0.9829
Learning Rate: 0.000003

Early stopping at epoch 101
Best F1: 0.9521 at epoch 86



Final Test Results:
Accuracy: 0.9473
F1-score: 0.9472
ROC-AUC: 0.9827
Precision: 0.9471
Recall: 0.9473
Best epoch: 86
